# Pokemon Legendary Predictor

This is a brief notebook detailing the implementation of a neural network built in Keras, a Python deep learning library, that aims to predict a pokemon's status as legendary or not (a binary classification problem) from a pokemon's in-game base stat values.

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
import pandas as pd
from keras import optimizers
import numpy as np
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold


# Read the data in
poke_data = pd.read_csv("pokemon.csv")


Using TensorFlow backend.
/home/vnxiclaire/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [9]:
# Look at the columns
poke_data.columns

Index(['abilities', 'against_bug', 'against_dark', 'against_dragon',
       'against_electric', 'against_fairy', 'against_fight', 'against_fire',
       'against_flying', 'against_ghost', 'against_grass', 'against_ground',
       'against_ice', 'against_normal', 'against_poison', 'against_psychic',
       'against_rock', 'against_steel', 'against_water', 'attack',
       'base_egg_steps', 'base_happiness', 'base_total', 'capture_rate',
       'classfication', 'defense', 'experience_growth', 'height_m', 'hp',
       'japanese_name', 'name', 'percentage_male', 'pokedex_number',
       'sp_attack', 'sp_defense', 'speed', 'type1', 'type2', 'weight_kg',
       'generation', 'is_legendary'],
      dtype='object')

In [3]:
# TODO: dummify the types so they can go into the net as well

#types = pd.get_dummies(poke_data["type1"])
#types2 = pd.get_dummies(poke_data["type2"])
#types2.columns = [x + "2" for x in types2]

In [4]:
# Extract the columns we need, and split the set (70% training, 30% test)

X = poke_data[['attack', 'defense', 'hp', 'sp_attack', 'sp_defense', 'speed']]

y = poke_data['is_legendary']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state = 420)



In [6]:
# Seed for reproducability
seed = 420


def create_model():
    
    # create model
    model = Sequential()
    
    # Add a layer, using sigmoid as the activation function
    model.add(Dense(1, input_shape=(6,),kernel_initializer='normal', activation='sigmoid'))
    

    # Train with stohastic gradient descent with a learning rate of .001
    opt = optimizers.SGD(lr=0.001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model



# Make the net, train the model
net = create_model()
net.fit(X_train,y_train,batch_size = 10, epochs=100)

Epoch 1/100
560/560 [==============================] - 0s 320us/step - loss: 1.3030 - acc: 0.8714
Epoch 2/100
560/560 [==============================] - 0s 213us/step - loss: 1.4232 - acc: 0.9107
Epoch 3/100
560/560 [==============================] - 0s 380us/step - loss: 1.4222 - acc: 0.9107
Epoch 4/100
560/560 [==============================] - 0s 295us/step - loss: 1.4567 - acc: 0.8982
Epoch 5/100
560/560 [==============================] - 0s 381us/step - loss: 1.4244 - acc: 0.9107
Epoch 6/100
560/560 [==============================] - 0s 257us/step - loss: 1.4225 - acc: 0.9107
Epoch 7/100
560/560 [==============================] - 0s 384us/step - loss: 1.2484 - acc: 0.8821
Epoch 8/100
560/560 [==============================] - 0s 260us/step - loss: 1.3541 - acc: 0.8804
Epoch 9/100
560/560 [==============================] - 0s 182us/step - loss: 1.3854 - acc: 0.8893
Epoch 10/100
560/560 [==============================] - 0s 195us/step - loss: 1.4197 - acc: 0.9000
Epoch 11/100
560/56

560/560 [==============================] - 0s 228us/step - loss: 1.4033 - acc: 0.9107
Epoch 84/100
560/560 [==============================] - 0s 363us/step - loss: 1.4203 - acc: 0.8911
Epoch 85/100
560/560 [==============================] - 0s 175us/step - loss: 1.3798 - acc: 0.8964
Epoch 86/100
560/560 [==============================] - 0s 328us/step - loss: 1.0568 - acc: 0.8679
Epoch 87/100
560/560 [==============================] - 0s 269us/step - loss: 1.4294 - acc: 0.9107
Epoch 88/100
560/560 [==============================] - 0s 154us/step - loss: 1.4282 - acc: 0.9107
Epoch 89/100
560/560 [==============================] - 0s 329us/step - loss: 1.4270 - acc: 0.9107
Epoch 90/100
560/560 [==============================] - 0s 279us/step - loss: 1.4258 - acc: 0.9107
Epoch 91/100
560/560 [==============================] - 0s 197us/step - loss: 1.4702 - acc: 0.8857
Epoch 92/100
560/560 [==============================] - 0s 393us/step - loss: 1.4293 - acc: 0.9107
Epoch 93/100
560/560 [=

Our accuracy seems to converge at around 91.07%, which is pretty good. But the accuracy seems to jump up and down ever so slightly, so we're probably overfitting somewhere.

## Cross Validation

We can perform stratified k-fold cross validation to evaluation the net's performance on the test set.

In [7]:
# Cross Validation
estimator = KerasClassifier(build_fn=create_model, nb_epoch=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X_test, y_test, cv=kfold)
print("Accuracy on the Test Set: %.2f%% (%.2f%%)" % (results.mean() * 100, results.std() * 100))

Accuracy on the Test Set: 91.28% (1.27%)


Yay, relatively similar performance on the test set!